In [1]:
#using soup environment, python version 3.7.0
import numpy as np
import matplotlib.pyplot as plt

from utils.myutils import *
from pathlib import Path
from resize.scipy import resize
import nibabel as nib

import pandas as pd

In [2]:
#----load images----- subject:04-skip-00/sub-OAS30167_ses-d0111_T1w

In [3]:
df = pd.DataFrame(columns=['Name','PSNR','SSIM','VIF','cpbd'])

In [4]:
#HR
hr_path ="/iacl/pg22/sam/data/OASIS3/OASIS3_iso/sub-OAS30167_ses-d0111_T1w.nii.gz"
hr = nib.load(hr_path)
hr_img = hr.get_fdata(dtype=np.float32)
hr_slice = hr_img[88,:,:]

In [5]:
lr_path ="/iacl/pg22/sam/data/OASIS3/OASIS3_LR/04-skip-00/sub-OAS30167_ses-d0111_T1w.nii.gz"
lr = nib.load(lr_path)
lr_img = lr.get_fdata(dtype=np.float32)

ratio = 256/64
lr_slice_upsampled = resize(lr_img[88,:,:], dxyz=(1,1/ratio), order=3) #upscale by 4, order = 3
lr_upsampled = resize(lr_img,dxyz=(1,1,1/ratio), order=3 )

In [6]:
#synthSR output
path = "./SynthSR/”synthSR_out”.nii"
obj = nib.load(path)
synthSR = obj.get_fdata(dtype=np.float32)
#WARNING: reshape synthSR
synthSR_resized = synthSR[:,:,0:256]

In [7]:
#smore
ash_dir = Path("/iacl/pg22/ashley/data/results/04-skip-00/sub-OAS30167_ses-d0111_T1w")
sm_img_path = ash_dir/"sub-OAS30167_ses-d0111_T1w_smore3.nii.gz"
obj3 = nib.load(sm_img_path)
sm_img = obj3.get_fdata(dtype=np.float32)
sm_slice = sm_img[88,:,:]


In [8]:
#kevin soup gan is a np array of the 3D brain volume
soup_arr = np.load("04-skip-00-sub-OAS30167_ses-d0111_T1w-kevinSR.npy")

In [9]:
#try VIF, FSIM, (NQM), CPBD, (LPC-SI), (FID), NIQE, (MMD), PSNR, SSIM, and the new frequency power method

In [10]:
#other methods (PSNR, SSIM, ISSM, FSIM)(this package also includes RMSE, SRE, SAM, and UIQ)
import image_similarity_measures
from image_similarity_measures.quality_metrics import ssim, psnr,issm,fsim

In [11]:
#import vif
from vif.vif_utils import vif
#vif(hr_img[88,:,:],sm_slice)

In [12]:
#cpbd only works with 2D images and with values from (0-255)
#this is a non-reference image quality measurement

import cpbd


In [13]:
#calculate vif for 3D volumes, returns the average of slices
def vif3d(vol1,vol2):
    score = 0
    for i in range(vol1.shape[0]):
        score += vif(vol1[i,:,:],vol2[i,:,:])
    
    final = score / vol1.shape[0]
    return final

In [14]:
#calculate vif for 3D volumes, returns the average of the slices
def cpbd3d(vol):
    score = 0
    for i in range(vol.shape[0]):
        score += cpbd.compute(vol[i,:,:])
    final = score / vol.shape[0]
    return final

In [67]:
#def freq_pwr(vol):
    
    #return (inner, outer)

In [15]:
#un-normalized images
dict = {'Name':'GT','PSNR':'NA','SSIM':'NA','VIF':'NA','cpbd':cpbd3d(normalize(hr_img,255))}

df = df.append(dict, ignore_index = True)


In [16]:
dict = {'Name':'Interpolated','PSNR':psnr(hr_img, lr_upsampled),'SSIM':ssim(hr_img, lr_upsampled),
        'VIF':vif3d(hr_img, lr_upsampled),'cpbd':cpbd3d(normalize(lr_upsampled,255))}
df = df.append(dict, ignore_index = True)
                    
dict = {'Name':'Smore','PSNR':psnr(hr_img, sm_img),'SSIM':ssim(hr_img, sm_img),
        'VIF':vif3d(hr_img, sm_img),'cpbd':cpbd3d(normalize(sm_img,255))}
df = df.append(dict, ignore_index = True)
                    
dict = {'Name':'Soup','PSNR':psnr(hr_img, soup_arr),'SSIM':ssim(hr_img, soup_arr),
        'VIF':vif3d(hr_img, soup_arr),'cpbd':cpbd3d(normalize(soup_arr,255))}
df = df.append(dict, ignore_index = True)


dict = {'Name':'SynthSR','PSNR':psnr(hr_img, synthSR_resized),'SSIM':ssim(hr_img,synthSR_resized),
        'VIF':vif3d(hr_img, synthSR_resized),'cpbd':cpbd3d(normalize(synthSR_resized,255))}
df = df.append(dict, ignore_index = True)


/home/ashley/miniconda3/envs/soup/lib/python3.7/site-packages/image_similarity_measures/quality_metrics.py:197: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return structural_similarity(org_img, pred_img, data_range=max_p, multichannel=True)


In [17]:
df

,Name,PSNR,SSIM,VIF,cpbd
0,GT,NA,NA,NA,0.709286
1,Interpolated,44.0223,0.972035,0.600754,0.008143
2,Smore,45.533,0.980694,0.693891,0.124268
3,Soup,8.24501,0.25017,0.560005,0.299377
4,SynthSR,31.8923,0.776052,0.115372,0.124220


In [19]:
#does this normalization improve the quality of the ouput images?
#if we only use it on soup_gan or synthSR, then we can say we've enhanced their performance
#do we also normalize the original HR image?
from intensity_normalization.normalize.nyul import NyulNormalize

In [21]:
#normalize to 20 04-skip-00 low resolution images
#warning: "Data contains negative values"
img_path =Path("/iacl/pg22/sam/data/OASIS3/OASIS3_LR/04-skip-00")
img_paths = list(img_path.iterdir())
images = []
for i in range(0,20):
    images.append(nib.load(img_paths[i]))

nyul_normalizer = NyulNormalize()
nyul_normalizer.fit(images) 
normalized = [nyul_normalizer(image) for image in images]
nyul_normalizer.save_standard_histogram("standard_histogram.npy")


/home/ashley/miniconda3/envs/soup/lib/python3.7/site-packages/intensity_normalization/normalize/base.py:171: UserWarning: Data contains negative values; skull-stripped functionality assumes the foreground is all positive. Provide the brain mask if otherwise.
  warnings.warn(msg)


In [22]:
#try to normalize soup_arr and synthSR
new_images = []
new_images.append(soup_arr)
new_images.append(synthSR_resized)
new_images.append(hr_img)
new_images.append(lr_upsampled)
new_images.append(sm_img)
normalized = [nyul_normalizer(image) for image in new_images]
n_soup,n_synthsr,n_hr,n_lr,n_sm = normalized[0],normalized[1],normalized[2],normalized[3],normalized[4]


In [23]:
n_df = pd.DataFrame(columns=['Name','PSNR','SSIM','VIF','cpbd'])


In [24]:
dict = {'Name':'GT','PSNR':'NA','SSIM':'NA','VIF':'NA','cpbd':cpbd3d(normalize(n_hr,255))}

n_df = n_df.append(dict, ignore_index = True)

dict = {'Name':'Interpolated','PSNR':psnr(n_hr, n_lr),'SSIM':ssim(n_hr, n_lr),
        'VIF':vif3d(n_hr, n_lr),'cpbd':cpbd3d(normalize(n_lr,255))}
n_df = n_df.append(dict, ignore_index = True)
                    
dict = {'Name':'Smore','PSNR':psnr(n_hr, n_sm),'SSIM':ssim(n_hr, n_sm),
        'VIF':vif3d(n_hr, n_sm),'cpbd':cpbd3d(normalize(n_sm,255))}
n_df = n_df.append(dict, ignore_index = True)
                    
dict = {'Name':'Soup','PSNR':psnr(n_hr, n_soup),'SSIM':ssim(n_hr, n_soup),
        'VIF':vif3d(n_hr, n_soup),'cpbd':cpbd3d(normalize(n_soup,255))}
n_df = n_df.append(dict, ignore_index = True)


dict = {'Name':'SynthSR','PSNR':psnr(n_hr, n_synthsr),'SSIM':ssim(n_hr, n_synthsr),
        'VIF':vif3d(n_hr, n_synthsr),'cpbd':cpbd3d(normalize(n_synthsr,255))}
n_df = n_df.append(dict, ignore_index = True)

/home/ashley/miniconda3/envs/soup/lib/python3.7/site-packages/image_similarity_measures/quality_metrics.py:197: FutureWarning: `multichannel` is a deprecated argument name for `structural_similarity`. It will be removed in version 1.0. Please use `channel_axis` instead.
  return structural_similarity(org_img, pred_img, data_range=max_p, multichannel=True)


In [25]:
n_df

,Name,PSNR,SSIM,VIF,cpbd
0,GT,NA,NA,NA,0.720351
1,Interpolated,57.1316,0.997032,0.744029,0.009395
2,Smore,58.6361,0.997977,0.842365,0.122641
3,Soup,56.8145,0.996708,0.63699,0.305655
4,SynthSR,43.4881,0.924478,0.173019,0.139890
